# Tax Credit Analysis: Zipcode-level Data Prep

## Setup

In [10]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
import jupyter_black

jupyter_black.load(
    lab=False,
    line_length=79,
    verbosity="DEBUG",
)

In [12]:
# Import packages
import os
import warnings

import geopandas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [13]:
warnings.filterwarnings("ignore")

In [14]:
# Prep demographic data
CSV_DIR = os.path.join("..", "csv_data")
FEATURE_DATA_PATH = os.path.join(CSV_DIR, "feature_data")
TAX_CREDIT_DATA_PATH = os.path.join(CSV_DIR, "tax_credit_data")

FIGURES_DIR = os.path.join("..", "figures")

demographics_colors = ["#1b9e77", "#7570b3", "#d95f02"]
pvi_colors = ["#e41a1c", "#984ea3", "#377eb8"]  # red, purple, blue

## Load in census - demographic data

### Race data

In [22]:

census2000_data_path = os.path.join(
    CSV_DIR,
    "DECENNIALSF12000.P008_2024-03-06T121235",
    "DECENNIALSF12000.P008-Data.csv",
)

census2010_data_path = os.path.join(
    CSV_DIR,
    "DECENNIALSF12010.P9_2023-10-03T093656",
    "DECENNIALSF12010.P9-Data.csv",
)
census2020_data_path = os.path.join(
    CSV_DIR,
    "DECENNIALDHC2020.P9_2023-10-03T092126",
    "DECENNIALDHC2020.P9-Data.csv",
)

In [28]:
census2000_data_df = pd.read_csv(census2000_data_path)
census2010_data_df = pd.read_csv(census2010_data_path)
census2020_data_df = pd.read_csv(census2020_data_path)

In [29]:
census2000_data_df

,Geography,Label for GEO_ID,Total,TotalNot Hispanic or Latino,TotalNot Hispanic or LatinoWhite alone,TotalNot Hispanic or LatinoBlack or African American alone,TotalNot Hispanic or LatinoAmerican Indian and Alaska Native alone,TotalNot Hispanic or LatinoAsian alone,TotalNot Hispanic or LatinoNative Hawaiian and Other Pacific Islander alone,TotalNot Hispanic or LatinoSome other race alone,TotalNot Hispanic or LatinoTwo or more races,TotalHispanic or Latino,TotalHispanic or LatinoWhite alone,TotalHispanic or LatinoBlack or African American alone,TotalHispanic or LatinoAmerican Indian and Alaska Native alone,TotalHispanic or LatinoAsian alone,TotalHispanic or LatinoNative Hawaiian and Other Pacific Islander alone,TotalHispanic or LatinoSome other race alone,TotalHispanic or LatinoTwo or more races,Unnamed: 19
0,8600000US00601,ZCTA5 00601,19143,66,61,3,0,1,0,0,1,19077,18154,227,31,1,0,378,286,NaN
1,8600000US00602,ZCTA5 00602,42042,295,244,15,0,10,2,1,23,41747,37036,1493,67,25,7,1977,1142,NaN
2,8600000US00603,ZCTA5 00603,55530,604,460,67,3,49,1,2,22,54926,45692,2747,115,66,13,4752,1541,NaN
3,8600000US00604,ZCTA5 00604,3923,515,421,37,0,28,0,7,22,3408,2941,123,4,2,0,249,89,NaN
4,8600000US00606,ZCTA5 00606,6449,56,52,1,1,1,0,0,1,6393,5772,193,9,0,0,204,215,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33173,8600000US99927,ZCTA5 99927,120,117,102,0,5,1,0,0,9,3,3,0,0,0,0,0,0,NaN
33174,8600000US99929,ZCTA5 99929,2424,2399,1790,3,360,13,3,1,229,25,7,0,3,2,0,9,4,NaN
33175,8600000US99950,ZCTA5 99950,36,34,31,0,3,0,0,0,0,2,2,0,0,0,0,0,0,NaN
33176,8600000US999HH,ZCTA5 999HH,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN


In [37]:
# census2000_data_df
census_2000_race_df = census2000_data_df[
    [
        "Geography",
        "Total",
        "TotalNot Hispanic or LatinoWhite alone",
    ]
]
census_2000_race_df["geoid"] = (
    census_2000_race_df["Geography"].astype(str).str[-5:]
)
census_2000_race_df.drop("Geography", axis=1, inplace=True)
census_2000_race_df.columns = ["total", "total_white_alone", "zipcode"]
census_2000_race_df["percent_white_alone_2000"] = census_2000_race_df[
    "total_white_alone"
].astype(float) / census_2000_race_df["total"].astype(float)
census_2000_race_df = census_2000_race_df[["zipcode", "percent_white_alone_2000"]]
census_2000_race_df


,zipcode,percent_white_alone_2000
0,00601,0.003187
1,00602,0.005804
2,00603,0.008284
3,00604,0.107316
4,00606,0.008063
...,...,...
33173,99927,0.850000
33174,99929,0.738449
33175,99950,0.861111
33176,999HH,NaN


In [38]:
# census2010_data_df
census_2010_race_df = census2010_data_df[
    [
        "Geography",
        "Total",
        "Total Not Hispanic or Latino Population of one race White alone",
    ]
]
census_2010_race_df["geoid"] = (
    census_2010_race_df["Geography"].astype(str).str[-5:]
)
census_2010_race_df.drop("Geography", axis=1, inplace=True)
census_2010_race_df.columns = ["total", "total_white_alone", "zipcode"]
census_2010_race_df["percent_white_alone_2010"] = census_2010_race_df[
    "total_white_alone"
].astype(float) / census_2010_race_df["total"].astype(float)
census_2010_race_df = census_2010_race_df[["zipcode", "percent_white_alone_2010"]]
census_2010_race_df

,zipcode,percent_white_alone_2010
0,00601,0.004308
1,00602,0.005202
2,00603,0.011483
3,00606,0.004837
4,00610,0.006445
...,...,...
33115,99923,0.908046
33116,99925,0.416361
33117,99926,0.098630
33118,99927,0.755319


In [39]:
census_2020_race_df = census2020_data_df[
    [
        "Geography",
        "Total:",
        "Total: Not Hispanic or Latino: Population of one race: White alone",
    ]
]
census_2020_race_df["geoid"] = (
    census_2020_race_df["Geography"].astype(str).str[-5:]
)
census_2020_race_df.drop("Geography", axis=1, inplace=True)
census_2020_race_df.columns = ["total", "total_white_alone", "zipcode"]
census_2020_race_df["percent_white_alone_2020"] = census_2020_race_df[
    "total_white_alone"
].astype(float) / census_2020_race_df["total"].astype(float)
census_2020_race_df = census_2020_race_df[["zipcode", "percent_white_alone_2020"]]
census_2020_race_df

,zipcode,percent_white_alone_2020
0,00601,0.002146
1,00602,0.005673
2,00603,0.011726
3,00606,0.005191
4,00610,0.006957
...,...,...
33769,99923,0.893939
33770,99925,0.373724
33771,99926,0.096671
33772,99927,0.857143


### Merge data and interpolate

In [42]:
tmp = pd.merge(
    census_2010_race_df, census_2020_race_df, on="zipcode", how="inner"
)
race_merged_df = pd.merge(
    census_2000_race_df, tmp, on="zipcode", how="inner"
)

race_merged_df

,zipcode,percent_white_alone_2000,percent_white_alone_2010,percent_white_alone_2020
0,00601,0.003187,0.004308,0.002146
1,00602,0.005804,0.005202,0.005673
2,00603,0.008284,0.011483,0.011726
3,00606,0.008063,0.004837,0.005191
4,00610,0.006077,0.006445,0.006957
...,...,...,...,...
31312,99923,0.948454,0.908046,0.893939
31313,99925,0.443979,0.416361,0.373724
31314,99926,0.096752,0.098630,0.096671
31315,99927,0.850000,0.755319,0.857143


In [44]:
race_interpolation_data = []

unique_zipcodes = race_merged_df["zipcode"].unique()
for zipcode in unique_zipcodes:
    
    value_2000 = race_merged_df[race_merged_df["zipcode"] == zipcode]["percent_white_alone_2000"].values[0]
    value_2010 = race_merged_df[race_merged_df["zipcode"] == zipcode]["percent_white_alone_2010"].values[0]
    value_2020 = race_merged_df[race_merged_df["zipcode"] == zipcode]["percent_white_alone_2020"].values[0]
    
    values_2000_2010 = np.linspace(value_2000, value_2010, 11)
    value_2009 = values_2000_2010[-2]
    race_interpolation_data.append([zipcode, 2009, value_2009])
    
    values_2010_2020 = np.linspace(value_2010, value_2020, 11)
    for ind, val in enumerate(values_2010_2020):
        year = 2010 + ind
        race_interpolation_data.append([zipcode, year, val])
    

In [45]:
race_interpolation_df = pd.DataFrame(race_interpolation_data, columns=["zipcode", "year", "percent_white_alone"])
race_interpolation_df

,zipcode,year,percent_white_alone
0,00601,2009,0.004196
1,00601,2010,0.004308
2,00601,2011,0.004092
3,00601,2012,0.003876
4,00601,2013,0.003659
...,...,...,...
375799,99929,2016,0.667126
375800,99929,2017,0.659195
375801,99929,2018,0.651264
375802,99929,2019,0.643333


### Median income data

In [46]:
acs2015_data_path = os.path.join(
    CSV_DIR,
    "ACSST5Y2015.S1901_2023-10-25T140904",
    "ACSST5Y2015.S1901-Data.csv",
)
acs2020_data_path = os.path.join(
    CSV_DIR,
    "ACSST5Y2020.S1901_2023-10-25T140841",
    "ACSST5Y2020.S1901-Data.csv",
)

In [47]:
acs2015_data_df = pd.read_csv(acs2015_data_path)
acs2020_data_df = pd.read_csv(acs2020_data_path)

In [48]:
acs2015_data_df["zipcode"] = acs2015_data_df["Geography"].astype(str).str[-5:]
acs2015_data_df = acs2015_data_df[
    ["zipcode", "Households Estimate Median income (dollars)"]
]
acs2015_data_df = acs2015_data_df.rename(
    {"Households Estimate Median income (dollars)": "median_income"}, axis=1
)
acs2015_data_df

,zipcode,median_income
0,00601,10816
1,00602,16079
2,00603,16804
3,00606,12512
4,00610,17475
...,...,...
33115,99923,-
33116,99925,38594
33117,99926,51071
33118,99927,19861


In [49]:
acs2020_data_df["zipcode"] = acs2020_data_df["Geography"].astype(str).str[-5:]
acs2020_data_df = acs2020_data_df[
    ["zipcode", "Estimate Households Median income (dollars)"]
]
acs2020_data_df = acs2020_data_df.rename(
    {"Estimate Households Median income (dollars)": "median_income"}, axis=1
)
acs2020_data_df

,zipcode,median_income
0,00601,14398
1,00602,16771
2,00603,15786
3,00606,14980
4,00610,20167
...,...,...
33115,99923,-
33116,99925,63125
33117,99926,58000
33118,99927,-


### Merge census demographic features

In [50]:
demographics_pre2015_df = acs2015_data_df.copy()
demographics_pre2015_df

,zipcode,median_income
0,00601,10816
1,00602,16079
2,00603,16804
3,00606,12512
4,00610,17475
...,...,...
33115,99923,-
33116,99925,38594
33117,99926,51071
33118,99927,19861


In [51]:
demographics_pre2015_df["median_income"] = pd.to_numeric(
    demographics_pre2015_df.median_income.astype(str).str.replace("-", ""),
    errors="coerce",
)

# demographics_pre2015_df["percent_white_alone"] = pd.to_numeric(
#     demographics_pre2015_df.percent_white_alone.astype(str).str.replace(
#         "-", ""
#     ),
#     errors="coerce",
# )
demographics_pre2015_df = demographics_pre2015_df.dropna()
demographics_pre2015_df

,zipcode,median_income
0,00601,10816.0
1,00602,16079.0
2,00603,16804.0
3,00606,12512.0
4,00610,17475.0
...,...,...
33114,99922,31250.0
33116,99925,38594.0
33117,99926,51071.0
33118,99927,19861.0


In [52]:
demographics_pre2020_df = acs2020_data_df.copy()
demographics_pre2020_df

,zipcode,median_income
0,00601,14398
1,00602,16771
2,00603,15786
3,00606,14980
4,00610,20167
...,...,...
33115,99923,-
33116,99925,63125
33117,99926,58000
33118,99927,-


In [53]:
demographics_pre2020_df["median_income"] = pd.to_numeric(
    demographics_pre2020_df.median_income.astype(str).str.replace("-", ""),
    errors="coerce",
)

# demographics_pre2020_df["percent_white_alone"] = pd.to_numeric(
#     demographics_pre2020_df.percent_white_alone.astype(str).str.replace(
#         "-", ""
#     ),
#     errors="coerce",
# )
demographics_pre2020_df = demographics_pre2020_df.dropna()
demographics_pre2020_df

,zipcode,median_income
0,00601,14398.0
1,00602,16771.0
2,00603,15786.0
3,00606,14980.0
4,00610,20167.0
...,...,...
33113,99921,54549.0
33114,99922,38750.0
33116,99925,63125.0
33117,99926,58000.0


## Tax Credit Data

### Read in and prep the tax credit data

In [54]:
def prep_agi_data(filepath, year):
    tax_credit_df = pd.read_csv(
        filepath,
        delimiter=",",
    )

    # Filter out state-level data
    if "ZIPCODE" in tax_credit_df.columns:
        tax_credit_df = tax_credit_df.rename(columns={"ZIPCODE": "zipcode"})
    if "AGI_STUB" in tax_credit_df.columns:
        tax_credit_df = tax_credit_df.rename(columns={"AGI_STUB": "agi_stub"})

    tax_credit_df = tax_credit_df[
        (tax_credit_df["zipcode"] != 0) & (tax_credit_df["zipcode"] != 99999)
    ]
    tax_credit_df = tax_credit_df[(tax_credit_df["agi_stub"] != "**")]

    tax_credit_df = tax_credit_df.rename(
        columns={
            "N07260": "Number of returns",
            "A07260": "Amount of returns",
            "N18500": "Number of real estate taxes",
            "N1": "Total Returns",
        }
    )
    tax_credit_df = tax_credit_df[tax_credit_df["Total Returns"] >= 1]

    # Filter down to only columns we need
    tax_credit_df = tax_credit_df[
        [
            "STATEFIPS",
            "STATE",
            "zipcode",
            "agi_stub",
            "Total Returns",
            "Number of returns",
            "Amount of returns",
            "Number of real estate taxes",
        ]
    ]

    # Add variables with varying normalization techniques
    tax_credit_df["year"] = year
    tax_credit_df["zipcode"] = (
        tax_credit_df["zipcode"].astype(str).str.zfill(5)
    )

    tax_credit_df["value_returns"] = 0
    tax_credit_df.loc[
        tax_credit_df["Number of returns"] >= 1, "value_returns"
    ] = (
        tax_credit_df["Amount of returns"].astype(float)
        / tax_credit_df["Number of returns"].astype(float)
    ) * 1000
    #     tax_credit_df["value_returns"] = (
    #         tax_credit_df["Amount of returns"] / tax_credit_df["Number of returns"]
    #     ) * 1000

    # Normalizing Amount of returns by investment from federal gov that year
    #     total_yearly_investiment = tax_credit_df["Amount of returns"].sum()
    #     tax_credit_df["fraction_of_federal_investment"] = (
    #         tax_credit_df["Amount of returns"] / total_yearly_investiment
    #     )

    # Normalizing Amount of returns by investment from federal gov that year
    # And with number of recipients in the community
    #     tax_credit_df["fraction_of_federal_investment_normalized"] = (
    #         tax_credit_df["value_returns"] / total_yearly_investiment
    #     )

    # Calculate participation
    tax_credit_df["percent_returns"] = 0
    tax_credit_df.loc[
        tax_credit_df["Number of returns"] >= 1, "percent_returns"
    ] = (
        tax_credit_df["Number of returns"].astype(float)
        / tax_credit_df["Total Returns"].astype(float)
    ) * 100

    # Normalizing participation by community by participation by state that year
    state_participation_totals = {}
    states = list(tax_credit_df["STATE"].unique())
    for state in states:
        state_participation = tax_credit_df[tax_credit_df["STATE"] == state][
            "Number of returns"
        ].sum()
        total_tax_returns = tax_credit_df[tax_credit_df["STATE"] == state][
            "Total Returns"
        ].sum()
        state_participation_totals[state] = (
            state_participation / total_tax_returns * 100
        )

    tax_credit_df["state_participation"] = tax_credit_df["STATE"].apply(
        lambda x: state_participation_totals.get(x)
    )

    tax_credit_df["relative_to_state_participation"] = (
        tax_credit_df["percent_returns"] / tax_credit_df["state_participation"]
    )

    tax_credit_df["percent_homeowners"] = 0
    tax_credit_df.loc[
        tax_credit_df["Number of real estate taxes"] >= 1, "percent_homeowners"
    ] = (
        tax_credit_df["Number of real estate taxes"]
        / tax_credit_df["Total Returns"]
    )

    return tax_credit_df

In [55]:
_2009 = prep_agi_data(
    os.path.join(TAX_CREDIT_DATA_PATH, "zipcode_agi", "09zpallagi.csv"), 2009
)
_2010 = prep_agi_data(
    os.path.join(TAX_CREDIT_DATA_PATH, "zipcode_agi", "10zpallagi.csv"), 2010
)

In [56]:
_2011 = prep_agi_data(
    os.path.join(TAX_CREDIT_DATA_PATH, "zipcode_agi", "11zpallagi.csv"), 2011
)
_2012 = prep_agi_data(
    os.path.join(TAX_CREDIT_DATA_PATH, "zipcode_agi", "12zpallagi.csv"), 2012
)
_2013 = prep_agi_data(
    os.path.join(TAX_CREDIT_DATA_PATH, "zipcode_agi", "13zpallagi.csv"), 2013
)
_2014 = prep_agi_data(
    os.path.join(TAX_CREDIT_DATA_PATH, "zipcode_agi", "14zpallagi.csv"), 2014
)
_2015 = prep_agi_data(
    os.path.join(TAX_CREDIT_DATA_PATH, "zipcode_agi", "15zpallagi.csv"), 2015
)

In [57]:
_2016 = prep_agi_data(
    os.path.join(TAX_CREDIT_DATA_PATH, "zipcode_agi", "16zpallagi.csv"), 2016
)
_2017 = prep_agi_data(
    os.path.join(TAX_CREDIT_DATA_PATH, "zipcode_agi", "17zpallagi.csv"), 2017
)
_2018 = prep_agi_data(
    os.path.join(TAX_CREDIT_DATA_PATH, "zipcode_agi", "18zpallagi.csv"), 2018
)
_2019 = prep_agi_data(
    os.path.join(TAX_CREDIT_DATA_PATH, "zipcode_agi", "19zpallagi.csv"), 2019
)
_2020 = prep_agi_data(
    os.path.join(TAX_CREDIT_DATA_PATH, "zipcode_agi", "20zpallagi.csv"), 2020
)

In [58]:
len(_2016["zipcode"].unique())

29872

In [59]:
# join all years together
pre2015_frames = [
    _2009,
    _2010,
    _2011,
    _2012,
    _2013,
    _2014,
    _2015,
]

pre2020_frames = [
    _2016,
    _2017,
    _2018,
    _2019,
    _2020,
]

### Combine tax data with demographics data

In [60]:
pre2015_tax_df = pd.concat(pre2015_frames)
pre2020_tax_df = pd.concat(pre2020_frames)

In [62]:
pre2015_frames_with_census = pd.merge(
    pre2015_tax_df, demographics_pre2015_df, on="zipcode"
)

pre2020_frames_with_census = pd.merge(
    pre2020_tax_df, demographics_pre2020_df, on="zipcode"
)

## Combine all years

In [65]:
credits_income_demographics_df = pd.concat(
    [pre2015_frames_with_census, pre2020_frames_with_census]
)
credits_income_demographics_df

,STATEFIPS,STATE,zipcode,agi_stub,Total Returns,Number of returns,Amount of returns,Number of real estate taxes,year,value_returns,percent_returns,state_participation,relative_to_state_participation,percent_homeowners,median_income
0,1,AL,35004,1,1468.0,0.0001,0.0001,112.0,2009,0.000000,0.000000,5.108432,0.000000,0.076294,62281.0
1,1,AL,35004,2,1271.0,82.0000,45.0000,450.0,2009,548.780488,6.451613,5.108432,1.262934,0.354052,62281.0
2,1,AL,35004,3,911.0,73.0000,58.0000,524.0,2009,794.520548,8.013172,5.108432,1.568617,0.575192,62281.0
3,1,AL,35004,4,552.0,63.0000,36.0000,398.0,2009,571.428571,11.413043,5.108432,2.234158,0.721014,62281.0
4,1,AL,35004,5,371.0,41.0000,31.0000,298.0,2009,756.097561,11.051213,5.108432,2.163328,0.803235,62281.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
754261,53,WA,98833,5,40.0,0.0000,0.0000,0.0,2020,0.000000,0.000000,0.964554,0.000000,0.000000,69615.0
754262,53,WA,99147,1,20.0,0.0000,0.0000,0.0,2020,0.000000,0.000000,0.964554,0.000000,0.000000,70208.0
754263,53,WA,99147,2,30.0,0.0000,0.0000,0.0,2020,0.000000,0.000000,0.964554,0.000000,0.000000,70208.0
754264,53,WA,99147,3,30.0,0.0000,0.0000,0.0,2020,0.000000,0.000000,0.964554,0.000000,0.000000,70208.0


### Merge with interpolated race data

In [66]:
credits_demographics_df = pd.merge(credits_income_demographics_df, race_interpolation_df, on=["zipcode", "year"])
credits_demographics_df

,STATEFIPS,STATE,zipcode,agi_stub,Total Returns,Number of returns,Amount of returns,Number of real estate taxes,year,value_returns,percent_returns,state_participation,relative_to_state_participation,percent_homeowners,median_income,percent_white_alone
0,1,AL,35004,1,1468.0,0.0001,0.0001,112.0,2009,0.000000,0.000000,5.108432,0.000000,0.076294,62281.0,0.865153
1,1,AL,35004,2,1271.0,82.0000,45.0000,450.0,2009,548.780488,6.451613,5.108432,1.262934,0.354052,62281.0,0.865153
2,1,AL,35004,3,911.0,73.0000,58.0000,524.0,2009,794.520548,8.013172,5.108432,1.568617,0.575192,62281.0,0.865153
3,1,AL,35004,4,552.0,63.0000,36.0000,398.0,2009,571.428571,11.413043,5.108432,2.234158,0.721014,62281.0,0.865153
4,1,AL,35004,5,371.0,41.0000,31.0000,298.0,2009,756.097561,11.051213,5.108432,2.163328,0.803235,62281.0,0.865153
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1746218,51,VA,22729,2,20.0,0.0000,0.0000,0.0,2020,0.000000,0.000000,1.339147,0.000000,0.000000,76667.0,0.511745
1746219,51,VA,22729,3,30.0,0.0000,0.0000,0.0,2020,0.000000,0.000000,1.339147,0.000000,0.000000,76667.0,0.511745
1746220,53,WA,98833,1,40.0,0.0000,0.0000,0.0,2020,0.000000,0.000000,0.964554,0.000000,0.000000,69615.0,0.891667
1746221,53,WA,98833,3,30.0,0.0000,0.0000,0.0,2020,0.000000,0.000000,0.964554,0.000000,0.000000,69615.0,0.891667


In [67]:
credits_demographics_df["percent_white_alone"] = (
    credits_demographics_df["percent_white_alone"] * 100
)
credits_demographics_df["percent_homeowners"] = (
    credits_demographics_df["percent_homeowners"] * 100
)

credits_demographics_df = credits_demographics_df[
    credits_demographics_df["percent_white_alone"] <= 100
]
credits_demographics_df = credits_demographics_df[
    credits_demographics_df["percent_homeowners"] <= 100
]

In [68]:
# Stackoverflow article for reference: https://stackoverflow.com/questions/56441190/zero-padding-pandas-column
credits_demographics_df["zipcode"] = (
    credits_demographics_df["zipcode"].astype(str).str.pad(5, fillchar="0")
)

In [69]:
credits_demographics_df.to_csv(
    os.path.join(
        CSV_DIR, "tax_credits_with_agi_demographic_zipcode_decade_updated_interpolated.csv"
    ),
    index=False,
)

In [70]:
len(credits_demographics_df["zipcode"].unique())

28722

In [73]:
100 - (28722 / 41704 * 100)

31.128908497985805

In [75]:
41704 - 28722

12982